In [ ]:
import numpy as np
import pandas as pd
import bayesian_modeling_functions as bm

import nest_asyncio # This package is required to run Stan package in Jupyter notebook.
nest_asyncio.apply()

# Recap of the Base Model
For all dates within a group, the following distributions hold: 

- $\theta_{t} \sim Gamma(\alpha^{\theta}, \beta^{\theta})$.
- $\sigma \sim Gamma(\alpha^{\sigma}, \beta^{\sigma})$.
- $X_{i,t} \sim TruncNormal(\theta_{t}, \sigma)$.

where $\theta_{t}$ is the mean Ct of all technical replicates on a given date pair, $\sigma$ is the common variance, and $X_{i,t}$ is the set of all replicate Ct values for the zone/date pair. Indices $i$ and $t$ represent replicate and date, respectively). 

# Learn the Prior with EM-MCMC
The first step is creating an instance of the `EMBayesianModel` class. We need to supply (1) `n1n2_xls_file_path`, the relative path to the data file containing the raw qPCR data, and (2) `censored_ct`, the maximum PCR cycle number of the experiment.

For instance, we may choose to specify the initial priors as: 
- $\sigma \sim Gamma(3, 1)$.
- $\theta_t \sim Gamma(1, \frac{1}{35})$.

This specification imposes a prior mean of 3 on $\sigma$ and a prior mean of 35 on the $\theta_t$s. If we plug these priors into the model and sample from it, the data will contain about $26\%$ non-detects. The initial prior selection should only affect the rate of convergence of the EM algorithm: as it runs, the algorithm should learn a set of reasonable priors.

In [ ]:
# hyper_par_dict = {"sigma": {"alpha": 3, "beta": 1}, "theta": {"alpha": 1, "beta": 1/35}}
hyper_par_dict = {"sigma": {"alpha": 3, "beta": 1}, "theta": {"alpha": 1, "beta": 1/35}}

em_bayesian_model = bm.EMBayesianModel(n1n2_xls_file_path="FILENAME.xls",
                                       censored_ct=45, hyper_par_dict=hyper_par_dict)

Next, we call the `em` method to run the EM-MCMC algorithm. 

`num_samples` determines the number of MCMC samples of each $\theta_i$ that will be generated at each iteration. For example, if `num_samples=10**4`, the model will generate $10^4$ MCMC samples for each $\theta_i$.

`drop_samples` is the number of MCMC samples ignored during the model update at each iteration. For example, when `num_samples=10**4` and `drop_samples=500`, the initial 500 out of $10^4$ MCMC samples will be ignored. Common practice is to fit the model on a smaller data subset and check the trace plot of the MCMC samples. Our tests on other WW data files indicate that the WW data model requires relatively few samples to "burn in". We can also check the trace plot of MCMC samples from the last iteration. 

`max_iteration` specifies how many times ($T$) the algorithm will iterate between the E-step (where the algorithm fits and samples from the posterior) and the M-step (where the algorithm updates the prior). We set `max_iteration` to be a large enough number so that the algorithm converges. We use the trace plots of posterior mean and hyperparameters ($\alpha_{\theta}$, $\beta_{\theta}$, etc.) to check algorithm convergence. In our tests, $T = 20$ was a large enough number to achieve convergence.

In [ ]:
em_bayesian_model.em(num_chains=1, num_samples=10**4, max_iteration=20, drop_samples=500)

In [ ]:
# Construct the trace plot of hyperparameters.
em_bayesian_model.em_trace_plot(type="hyper_parameter", figsize=(6, 4))

In [ ]:
# Construct the trace plot of the posterior means.
em_bayesian_model.em_trace_plot(type="posterior_mean", figsize=(20, 10))

`posterior_col_names` will export the column names of the posterior data frame produced by Stan. We extract the 
column names which we are interested in (sigma and thetas). 

In [ ]:
em_bayesian_model.posterior_col_names()

In [ ]:
par_name_list=em_bayesian_model.posterior_col_names()[7:]
par_name_list

We generate trace plots of MCMC samples from the last iteration. Plots have no obvious patterns, which is desirable.

In [ ]:
em_bayesian_model.diagnosis_plot(par_name_list=par_name_list, drop_samples=500, figsize=(20, 30))

We are now ready to extract the hyperparameters the algorithm learned and apply them to the new data.

In [ ]:
new_hyper_par_dict = em_bayesian_model.final_hyper_parameter()
new_hyper_par_dict

# Imputation on New Data

We again use the `EMBayesianModel` class to fit the model, but now we use the `new_hyper_par_dict` obtained in the previous section.

In [ ]:
new_em_bayesian_model = bm.EMBayesianModel(n1n2_xls_file_path="FILENAME.xls", censored_ct=45, hyper_par_dict=new_hyper_par_dict)

There are two options for imputation. One is that we again run the EM-MCMC for many iterations until the posterior mean is stable. The second option is fitting the model once, which is computationally much cheaper. To fit the model once, we set `max_iteration=1`.

In [ ]:
new_em_bayesian_model.em(num_chains=1, num_samples=10**4, max_iteration=1, drop_samples=500)

We again check trace plots of MCMC samples.

In [ ]:
par_name_list=new_em_bayesian_model.posterior_col_names()[7:]
par_name_list

In [ ]:
new_em_bayesian_model.diagnosis_plot(par_name_list=par_name_list, drop_samples=500, figsize=(20, 30))

Finally, we geenerate the summary of the inference.

In [ ]:
a = new_em_bayesian_model.par_summary(drop_samples=500, par_name_list=par_name_list)